In [1]:
!pip install -q pyiceberg

In [2]:
from pyiceberg.catalog.rest import RestCatalog
import logging

import pandas as pd
import pyarrow.parquet as pq
import pyarrow as pa

CATALOG_URL = "http://lakekeeper:8181/catalog"
DEMO_WAREHOUSE = "demo"

CLIENT_ID = "spark"
CLIENT_SECRET = "2OR3eRvYfSZzzZ16MlPd95jhLnOaLM52"

catalog = RestCatalog(
    name="demo",
    warehouse="demo",
    uri=CATALOG_URL,
    credential=f"{CLIENT_ID}:{CLIENT_SECRET}",
    **{"oauth2-server-uri": "http://keycloak:8080/realms/iceberg/protocol/openid-connect/token", "scope": "lakekeeper"},
)


In [3]:
import requests, jwt
from IPython.display import JSON

CATALOG_URL = "http://lakekeeper:8181/catalog"
MANAGEMENT_URL = "http://lakekeeper:8181/management"
KEYCLOAK_TOKEN_URL = "http://keycloak:8080/realms/iceberg/protocol/openid-connect/token"

In [4]:
# Login to Keycloak
CLIENT_ID = "spark"
CLIENT_SECRET = "2OR3eRvYfSZzzZ16MlPd95jhLnOaLM52"

response = requests.post(
    url=KEYCLOAK_TOKEN_URL,
    data={
        "grant_type": "client_credentials",
        "client_id": CLIENT_ID,
        "client_secret": CLIENT_SECRET,
        "scope": "lakekeeper"
    },
    headers={"Content-type": "application/x-www-form-urlencoded"},
)
response.raise_for_status()
access_token = response.json()['access_token']

# Lets inspect the token we got to see that our application name is available:
JSON(jwt.decode(access_token, options={"verify_signature": False}))

<IPython.core.display.JSON object>

# 1. create 100 namespace, create 50 table per namespace

In [5]:
for i in range(100):
  ns_name = f"namespace{i+1}"
  test_namespace = (ns_name,)
  if test_namespace not in catalog.list_namespaces():
    catalog.create_namespace(test_namespace)
  for j in range(50):
    test_table = (ns_name, f"my_table{j+1}")
    df = pd.DataFrame(
        {
            "id": [1, 2, 3],
            "data": ["a", "b", "c"],
        }
    )
    pa_df = pa.Table.from_pandas(df)
    table = catalog.create_table(
        test_table,
        schema=pa_df.schema,
    )

    # table.append(pa_df)

# 2. bulk grant

## 2.1 create utility functions

In [6]:
# utility functions
def find_warehouse_id(name):
    response = requests.get(
        url=f"{MANAGEMENT_URL}/v1/warehouse",
        headers={"Authorization": f"Bearer {access_token}"}
    )
    response.raise_for_status()
    warehouses = response.json()['warehouses']
    for warehouse in warehouses:
        if warehouse['name'] == name:
            return warehouse['id']
    return None
def find_table_id(warehouse_id, namespace, name):
    response = requests.get(
        url=f"{CATALOG_URL}/v1/{warehouse_id}/namespaces/{namespace}/tables/{name}",
        headers={"Authorization": f"Bearer {access_token}"}
    )
    response.raise_for_status()
    return response.json()['metadata']['table-uuid']
def assign_select(warehouse_id, table_id, user_id):
    response = requests.post(
        url=f"{MANAGEMENT_URL}/v1/permissions/warehouse/{warehouse_id}/table/{table_id}/assignments",
        headers={"Authorization": f"Bearer {access_token}"},
        json={"writes": [{"type": "select", "user": user_id}]}
    )
    response.raise_for_status()
def assign_table(warehouse, namespace, table, user):
    warehouse_id = find_warehouse_id(warehouse)
    table_id = find_table_id(warehouse_id, namespace, table)
    assign_select(warehouse_id, table_id, user)

## 2.2 login in

In [7]:
user_dict = {}
for client_id, client_secret in [("trino", "AK48QgaKsqdEpP9PomRJw7l2T7qWGHdZ"), ("duckdb", "r2dHUlb7XrkSRcvrRqG5XZwQfnUS5NlL"), ("starrocks", "X5IWbfDJBTcU1F3PGZWgxDJwLyuFQmSf")]:
    response = requests.post(
        url=KEYCLOAK_TOKEN_URL,
        data={
            "grant_type": "client_credentials",
            "client_id": client_id,
            "client_secret": client_secret,
            "scope": "lakekeeper"
        },
        headers={"Content-type": "application/x-www-form-urlencoded"},
    )
    response.raise_for_status()
    access_token_client = response.json()['access_token']
    
    response = requests.post(
        url=f"{MANAGEMENT_URL}/v1/user",
        headers={"Authorization": f"Bearer {access_token_client}"},
        json={"update-if-exists": True}
    )
    response.raise_for_status()
    user = jwt.decode(access_token_client, options={"verify_signature": False})
    user_dict[client_id] = user['sub']

# 2.3 grant

In [8]:
grants = [
    {"client": "trino", "namespaces": [1,2], "tables": range(50)},
    {"client": "duckdb", "namespaces": [5,6], "tables": range(20)},
    {"client": "starrocks", "namespaces": [10,11,12], "tables": range(30)},
]

for grant in grants:
    print(f"current: {grant['client']}")
    for i in grant["namespaces"]:
        for j in grant["tables"]:
            assign_table("demo", f"namespace{i}", f"my_table{j+1}", f"oidc~{user_dict[grant['client']]}")

current: trino
current: duckdb
current: starrocks
